In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#SK-Learn        
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

from pprint import pprint
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR,LinearSVR
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE, f_regression
from sklearn.model_selection import GridSearchCV

import time

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv
/kaggle/input/graduate-admissions/Admission_Predict.csv


In [2]:
PATH=Path("/kaggle/input/graduate-admissions")
path_11=PATH/"Admission_Predict_Ver1.1.csv"

In [3]:
df=pd.read_csv(path_11,index_col='Serial No.')

In [4]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1 to 500
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          500 non-null    int64  
 1   TOEFL Score        500 non-null    int64  
 2   University Rating  500 non-null    int64  
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
 7   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 35.2 KB


In [6]:
X=df.copy()
y=df['Chance of Admit ']
X.drop('Chance of Admit ',axis=1,inplace=True)

# **Splitting into training and test set**

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((420, 7), (80, 7), (420,), (80,))

* Final accuracy would be checked on Test Dataset(X_test, y_test)
* Cross valiation would be used as dataset is not very big

# **Feature Scaling**

In [8]:
mean = X_train.mean()
std = X_train.std()

In [9]:
X_train = (X_train-mean)/std
X_test = (X_test-mean)/std

In [10]:
print(std,mean)

GRE Score            11.457694
TOEFL Score           6.160395
University Rating     1.152560
SOP                   1.005567
LOR                   0.935844
CGPA                  0.604475
Research              0.498087
dtype: float64 GRE Score            316.130952
TOEFL Score          107.252381
University Rating      3.102381
SOP                    3.370238
LOR                    3.490476
CGPA                   8.568738
Research               0.550000
dtype: float64


In [11]:
X_train.head(3)

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
Serial No.,,,,,,,
261,0.948624,0.121359,1.646438,1.620740,0.010177,0.928511,0.903457
118,-2.280647,-0.527950,0.778804,-1.362652,-1.058377,-1.834217,-1.104225
188,1.646845,1.744631,1.646438,1.123508,0.010177,1.441353,0.903457


In [12]:
models = pd.DataFrame(columns=['model', 'score','Time to Train'])


options = [
           SVR(), 
           LinearSVR(),
           SGDRegressor(), 
           XGBRegressor()]   

model_names = [ 
               'Support Vector Machine', 
               'Linear SVC', 
               'SGD Regressor',
               'XGBoost']  

for (opt, name) in zip(options, model_names):
    start=time.time()
    model = opt
    print(model)
    model.fit(X_train, y_train)
    scores = model.score(X_test,y_test)
    end=time.time()
    row = pd.DataFrame([[name, scores.mean(), end-start]], columns=['model', 'score','Time to Train'])
    models = pd.concat([models, row], ignore_index=True)

models.sort_values(by='score', ascending=False)


SVR()
LinearSVR()
SGDRegressor()
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)


,model,score,Time to Train
2,SGD Regressor,0.805789,0.005520
1,Linear SVC,0.803990,0.019693
3,XGBoost,0.753161,0.071926
0,Support Vector Machine,0.736306,0.013678


* SGD Regressor has the best accuracy(L2 Regularization)

# Feature Engineering

In [13]:
model = SGDRegressor(random_state = 42)
model.fit(X_train, y_train)

SGDRegressor(random_state=42)

In [14]:
SGD = RFE(model, n_features_to_select=1, verbose =3)
SGD.fit(X_train,y_train)

imp1 = pd.DataFrame({'feature':X_train.columns, 'rank1':SGD.ranking_})
imp1 = imp1.sort_values(by = 'rank1')
imp1

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.


,feature,rank1
5,CGPA,1
0,GRE Score,2
1,TOEFL Score,3
4,LOR,4
6,Research,5
2,University Rating,6
3,SOP,7


In [15]:
df.corr()['Chance of Admit ']

GRE Score            0.810351
TOEFL Score          0.792228
University Rating    0.690132
SOP                  0.684137
LOR                  0.645365
CGPA                 0.882413
Research             0.545871
Chance of Admit      1.000000
Name: Chance of Admit , dtype: float64

* None of the feature has <0.5 correlation 
* Thus we would be keeping all the features

# **HyperParameter Tuning**

In [16]:
print('Parameters currently in use:\n')
pprint(SGD.get_params())

Parameters currently in use:

{'estimator': SGDRegressor(random_state=42),
 'estimator__alpha': 0.0001,
 'estimator__average': False,
 'estimator__early_stopping': False,
 'estimator__epsilon': 0.1,
 'estimator__eta0': 0.01,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.15,
 'estimator__learning_rate': 'invscaling',
 'estimator__loss': 'squared_loss',
 'estimator__max_iter': 1000,
 'estimator__n_iter_no_change': 5,
 'estimator__penalty': 'l2',
 'estimator__power_t': 0.25,
 'estimator__random_state': 42,
 'estimator__shuffle': True,
 'estimator__tol': 0.001,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'n_features_to_select': 1,
 'step': 1,
 'verbose': 3}


* learning rate : The constant multiplied to the error before correcting the weights.
* loss : The loss function to be used. The possible values are ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’.
* validation_fraction : The proportion of training data to set aside as validation set 
* penalty: The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. 

In [17]:
SGD_tmp=SGDRegressor(random_state=42)
param_grid={
    'loss': ['squared_loss','huber'],
    'penalty':['l1','l2'],
    'validation_fraction':[0.1,0.12,0.15],
    'learning_rate':['optimal','invscaling','adaptive']
}
pprint(param_grid)

{'learning_rate': ['optimal', 'invscaling', 'adaptive'],
 'loss': ['squared_loss', 'huber'],
 'penalty': ['l1', 'l2'],
 'validation_fraction': [0.1, 0.12, 0.15]}


In [18]:
SGD_grid=GridSearchCV(estimator=SGD_tmp,param_grid=param_grid,cv=5)
SGD_grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42),
             param_grid={'learning_rate': ['optimal', 'invscaling', 'adaptive'],
                         'loss': ['squared_loss', 'huber'],
                         'penalty': ['l1', 'l2'],
                         'validation_fraction': [0.1, 0.12, 0.15]})

In [19]:
SGD_grid.best_params_

{'learning_rate': 'adaptive',
 'loss': 'huber',
 'penalty': 'l2',
 'validation_fraction': 0.1}

In [20]:
SGD_tuned=SGDRegressor(loss='huber',penalty='l2',learning_rate='adaptive',validation_fraction=0.1)
SGD_tuned.fit(X_train,y_train)

SGDRegressor(learning_rate='adaptive', loss='huber')

In [21]:
print("Accuracy for Random Forest after Hyperparameter Tuning on test data: ",SGD_tuned.score(X_test,y_test))
print("Accuracy for Random Forest before Hyperparameter Tuning on test data: ",SGD.score(X_test,y_test))

Accuracy for Random Forest after Hyperparameter Tuning on test data:  0.8113202184112717
Accuracy for Random Forest before Hyperparameter Tuning on test data:  0.7797531222089786


# Saving the model

In [22]:
import pickle
Pkl_Filename = "Pickle_SGD_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(SGD_tuned, file)